In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [97]:
%load_ext autoreload
%autoreload 2
from scripts.extractive_summary.ltr.ltr_features_targets import LTRFeaturesTargets
from scripts.models.ltr.train import LTRTrain
from scripts.models.baseline_rank.baseline_rank import BaselineRank

from scripts.text.article_text_processor import ArticleTextProcessor

from scripts import conf

%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd

# Learning to Rank: salida ordenada

Una vez se ha obtenido un score que permita ordenar cada evento dentro
de un partido, se debe construir el "resumen" a partir de ello.

La aproximación más sencilla sería ordenar los eventos para cada partido, y escoger los N primeros eventos con mayor score para representar la noticia. Sin embargo, como ya hemos comentado con 
anterioridad, esto puede sufrir varios problemas: puede haber muchos
eventos con el mismo score, y además muchos eventos pueden estar
referidos a la misma frase del artículo.

# Baseline

Primero, debemos proporcionar un objeto que represente el sistema
que asigna scores a los eventos. Para obtener un primer resultado y validar el proceso, se usará directamente el score calculado con los procesos de targets de LTR, sin usar modelos. Este supondrá la cota máxima que podemos alcanzar usando los scores calculados.

In [5]:
key_events = ['goal', 'red_card', 'penalty']
lags = [1, 3, 5]
target_metric = 'rouge'
drop_teams = True
lemma = True
metric_params = {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'}
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}

In [6]:
ltr = LTRFeaturesTargets(target_metric=target_metric, 
                        key_events=key_events,
                        lags=lags,
                        metric_params=metric_params,
                        count_vec_kwargs=count_vec_kwargs,
                        drop_teams=drop_teams,
                        lemma=lemma)

Setting target metric to rouge


In [91]:
rank = BaselineRank(ltr=ltr, n=10)

Received LTR Features targets. Training and scoring are not necessary


In [24]:
df = rank.load_data()

In [25]:
df

,length,n_stop,is_key_event,n_players,players_importance,advantage,equalize,position,tfidf_sum,sim_previous_1,sim_previous_3,sim_previous_5,event_ix,url,json_file,sentence_ix,score
0,12,4,0,2,0.102041,0,0,0.012048,4.527906,0.000000,0.000000,0.000000,0,http://www.premierleague.com/match/38678,premier_league_2018_2019.json,4,0.153846
1,3,2,0,1,0.040816,0,0,0.024096,2.190114,0.000000,0.000000,0.000000,1,http://www.premierleague.com/match/38678,premier_league_2018_2019.json,3,0.181818
2,8,5,0,2,0.142857,0,0,0.036145,3.573445,0.000000,0.000000,0.000000,2,http://www.premierleague.com/match/38678,premier_league_2018_2019.json,2,0.222222
3,3,2,0,1,0.091837,0,0,0.048193,2.205765,0.241337,0.000000,0.000000,3,http://www.premierleague.com/match/38678,premier_league_2018_2019.json,0,0.000000
4,6,3,0,1,0.040816,0,0,0.060241,3.246406,0.000000,0.160309,0.000000,4,http://www.premierleague.com/match/38678,premier_league_2018_2019.json,5,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423795,3,1,0,1,0.000539,0,0,0.964286,2.178008,0.000000,0.000000,0.078282,107,https://www.espn.com/soccer/report?gameId=522804,italian_serie_a_2018_2019.json,0,0.000000
423796,4,0,0,2,0.000853,0,0,0.973214,2.602282,0.000000,0.000000,0.000000,108,https://www.espn.com/soccer/report?gameId=522804,italian_serie_a_2018_2019.json,3,0.133333
423797,14,7,0,2,0.000797,0,0,0.982143,4.892766,0.000000,0.000000,0.000000,109,https://www.espn.com/soccer/report?gameId=522804,italian_serie_a_2018_2019.json,7,0.181818
423798,8,4,0,2,0.001116,0,0,0.991071,3.609882,0.098261,0.000000,0.000000,110,https://www.espn.com/soccer/report?gameId=522804,italian_serie_a_2018_2019.json,1,0.090909


In [26]:
df_cols = df[['url', 'score']].copy()

In [27]:
df_cols

,url,score
0,http://www.premierleague.com/match/38678,0.153846
1,http://www.premierleague.com/match/38678,0.181818
2,http://www.premierleague.com/match/38678,0.222222
3,http://www.premierleague.com/match/38678,0.000000
4,http://www.premierleague.com/match/38678,0.200000
...,...,...
423795,https://www.espn.com/soccer/report?gameId=522804,0.000000
423796,https://www.espn.com/soccer/report?gameId=522804,0.133333
423797,https://www.espn.com/soccer/report?gameId=522804,0.181818
423798,https://www.espn.com/soccer/report?gameId=522804,0.090909


In [28]:
df_events_url = df_cols.groupby('url').count()

In [29]:
df_events_url

,score
url,
http://www.premierleague.com/match/14040,118
http://www.premierleague.com/match/14041,76
http://www.premierleague.com/match/14042,103
http://www.premierleague.com/match/14043,110
http://www.premierleague.com/match/14044,102
...,...
https://www.espn.com/soccer/report?gameId=522802,100
https://www.espn.com/soccer/report?gameId=522804,112
https://www.espn.com/soccer/report?gameId=522811,113


In [30]:
df_cols_copy = df_cols.copy()
df_rank = df_cols_copy.groupby('url').rank(method='max')
df_rank = df_rank.rename({'score': 'rank'}, axis=1)

In [31]:
df_rank

,rank
0,40.0
1,48.0
2,69.0
3,20.0
4,65.0
...,...
423795,29.0
423796,77.0
423797,99.0
423798,43.0


In [32]:
df_cols_rank = df_cols.join(df_rank)

In [33]:
df_cols_rank

,url,score,rank
0,http://www.premierleague.com/match/38678,0.153846,40.0
1,http://www.premierleague.com/match/38678,0.181818,48.0
2,http://www.premierleague.com/match/38678,0.222222,69.0
3,http://www.premierleague.com/match/38678,0.000000,20.0
4,http://www.premierleague.com/match/38678,0.200000,65.0
...,...,...,...
423795,https://www.espn.com/soccer/report?gameId=522804,0.000000,29.0
423796,https://www.espn.com/soccer/report?gameId=522804,0.133333,77.0
423797,https://www.espn.com/soccer/report?gameId=522804,0.181818,99.0
423798,https://www.espn.com/soccer/report?gameId=522804,0.090909,43.0


Con este método habría muchos scores iguales!! En media, el 88%
de los eventos de cada partido tendría el mismo ranking...

In [55]:
df_urls_rank = df_cols_rank.groupby('url').agg({'score': 'count', 'rank': 'nunique'})
df_urls_rank['perc'] = df_urls_rank['rank']/df_urls_rank['score']*100

In [57]:
df_urls_rank['perc'].describe()

count    4523.000000
mean       12.823766
std         6.192692
min         1.020408
25%         9.643839
50%        12.149533
75%        15.000000
max       100.000000
Name: perc, dtype: float64

In [58]:
df_urls_rank

,score,rank,perc
url,,,
http://www.premierleague.com/match/14040,118,16,13.559322
http://www.premierleague.com/match/14041,76,14,18.421053
http://www.premierleague.com/match/14042,103,12,11.650485
http://www.premierleague.com/match/14043,110,19,17.272727
http://www.premierleague.com/match/14044,102,16,15.686275
...,...,...,...
https://www.espn.com/soccer/report?gameId=522802,100,5,5.000000
https://www.espn.com/soccer/report?gameId=522804,112,17,15.178571
https://www.espn.com/soccer/report?gameId=522811,113,14,12.389381


Para esta primera aproximación, evitaremos duplicados utilizando
la opción first de rank de pandas (equivalente a row_number de sql)

In [34]:
df_cols_copy = df_cols.copy()
df_rank = df_cols_copy.groupby('url').rank(method='first', ascending=False)
df_rank = df_rank.rename({'score': 'rank'}, axis=1)

In [35]:
df_cols_rank = df_cols.join(df_rank)

In [101]:
df_cols_rank.groupby('url').agg({'score': 'count', 'rank': 'nunique'})

,score,rank
url,,
http://www.premierleague.com/match/14040,118,118
http://www.premierleague.com/match/14041,76,76
http://www.premierleague.com/match/14042,103,103
http://www.premierleague.com/match/14043,110,110
http://www.premierleague.com/match/14044,102,102
...,...,...
https://www.espn.com/soccer/report?gameId=522802,100,100
https://www.espn.com/soccer/report?gameId=522804,112,112
https://www.espn.com/soccer/report?gameId=522811,113,113


In [36]:
df_cols_rank[df_cols_rank.url=='http://www.premierleague.com/match/14040']

,url,score,rank
160232,http://www.premierleague.com/match/14040,1.000000,1.0
160233,http://www.premierleague.com/match/14040,0.250000,53.0
160234,http://www.premierleague.com/match/14040,0.333333,19.0
160235,http://www.premierleague.com/match/14040,0.333333,20.0
160236,http://www.premierleague.com/match/14040,0.142857,72.0
...,...,...,...
160345,http://www.premierleague.com/match/14040,0.117647,79.0
160346,http://www.premierleague.com/match/14040,0.076923,87.0
160347,http://www.premierleague.com/match/14040,0.666667,13.0
160348,http://www.premierleague.com/match/14040,0.333333,52.0


In [37]:
df_cols_rank[df_cols_rank['rank'] <= 10]

,url,score,rank
11,http://www.premierleague.com/match/38678,0.300000,8.0
21,http://www.premierleague.com/match/38678,0.444444,1.0
22,http://www.premierleague.com/match/38678,0.363636,6.0
31,http://www.premierleague.com/match/38678,0.400000,2.0
33,http://www.premierleague.com/match/38678,0.400000,3.0
...,...,...,...
423745,https://www.espn.com/soccer/report?gameId=522804,0.214286,9.0
423746,https://www.espn.com/soccer/report?gameId=522804,0.285714,3.0
423771,https://www.espn.com/soccer/report?gameId=522804,0.266667,7.0
423777,https://www.espn.com/soccer/report?gameId=522804,0.214286,10.0


## Juntando con eventos reales

El siguiente paso será juntar los rankings con los eventos reales, 
para poder generar el resumen que se comparará con el artículo.

In [71]:
rank_df = rank.rank_events()
events_df = rank._create_events_df()

Ranking events...


In [72]:
rank_df

,url,event_ix,score,rank
11,http://www.premierleague.com/match/38678,11,0.300000,8.0
21,http://www.premierleague.com/match/38678,21,0.444444,1.0
22,http://www.premierleague.com/match/38678,22,0.363636,6.0
31,http://www.premierleague.com/match/38678,31,0.400000,2.0
33,http://www.premierleague.com/match/38678,33,0.400000,3.0
...,...,...,...,...
423745,https://www.espn.com/soccer/report?gameId=522804,57,0.214286,9.0
423746,https://www.espn.com/soccer/report?gameId=522804,58,0.285714,3.0
423771,https://www.espn.com/soccer/report?gameId=522804,83,0.266667,7.0
423777,https://www.espn.com/soccer/report?gameId=522804,89,0.214286,10.0


In [73]:
events_df

,url,event_ix,event
0,http://www.premierleague.com/match/38678,0,Attempt blocked. Ilkay Gündogan (Manchester Ci...
1,http://www.premierleague.com/match/38678,1,"Corner, Brighton and Hove Albion. Conceded by..."
2,http://www.premierleague.com/match/38678,2,"Offside, Brighton and Hove Albion. Anthony Kno..."
3,http://www.premierleague.com/match/38678,3,Foul by Anthony Knockaert (Brighton and Hove A...
4,http://www.premierleague.com/match/38678,4,Aymeric Laporte (Manchester City) wins a free ...
...,...,...,...
423795,https://www.espn.com/soccer/report?gameId=522804,107,Foul by Marcelo Brozovic (Inter Milan).
423796,https://www.espn.com/soccer/report?gameId=522804,108,"Substitution, Inter Milan. Lautaro Martínez re..."
423797,https://www.espn.com/soccer/report?gameId=522804,109,Attempt saved. Ivan Perisic (Inter Milan) left...
423798,https://www.espn.com/soccer/report?gameId=522804,110,"Offside, Inter Milan. Ivan Perisic tries a thr..."


In [74]:
rank_events_df = rank_df.merge(events_df, on=['url', 'event_ix'], how='inner')

In [75]:
rank_events_df

,url,event_ix,score,rank,event
0,http://www.premierleague.com/match/38678,11,0.300000,8.0,Attempt saved. Riyad Mahrez (Manchester City) ...
1,http://www.premierleague.com/match/38678,21,0.444444,1.0,"Goal. Brighton and Hove Albion 1, Manchester ..."
2,http://www.premierleague.com/match/38678,22,0.363636,6.0,"Goal. Brighton and Hove Albion 1, Manchester ..."
3,http://www.premierleague.com/match/38678,31,0.400000,2.0,Attempt saved. Riyad Mahrez (Manchester City) ...
4,http://www.premierleague.com/match/38678,33,0.400000,3.0,"Goal. Brighton and Hove Albion 1, Manchester ..."
...,...,...,...,...,...
45091,https://www.espn.com/soccer/report?gameId=522804,57,0.214286,9.0,Iago Falque (Torino) wins a free kick in the d...
45092,https://www.espn.com/soccer/report?gameId=522804,58,0.285714,3.0,"Goal! Inter Milan 2, Torino 1. Andrea Belotti..."
45093,https://www.espn.com/soccer/report?gameId=522804,83,0.266667,7.0,Stefan de Vrij (Inter Milan) wins a free kick ...
45094,https://www.espn.com/soccer/report?gameId=522804,89,0.214286,10.0,Attempt missed. Andrea Belotti (Torino) header...


In [84]:
match_rank_df

,event
url,
http://www.premierleague.com/match/14040,"[Offside, Liverpool. Adam Lallana tries a thro..."
http://www.premierleague.com/match/14041,[Delay in match Simon Francis (Bournemouth) be...
http://www.premierleague.com/match/14042,"[Foul by Federico Fernández (Swansea City)., A..."
http://www.premierleague.com/match/14043,[Diego Costa (Chelsea) wins a free kick in the...
http://www.premierleague.com/match/14044,[Attempt blocked. Salomón Rondón (West Bromwic...
...,...
https://www.espn.com/soccer/report?gameId=522802,[Attempt missed. Piotr Zielinski (Napoli) left...
https://www.espn.com/soccer/report?gameId=522804,"[Goal! Inter Milan 1, Torino 0. Ivan Perisic ..."
https://www.espn.com/soccer/report?gameId=522811,[Attempt missed. Franck Kessié (Milan) right f...


In [87]:
match_rank_df = rank_events_df.groupby('url').agg({'event': list})
match_rank_df = match_rank_df['event'].apply(lambda l: ' '.join(l))

In [88]:
match_rank_df.loc['http://www.premierleague.com/match/38678']

'Attempt saved. Riyad Mahrez (Manchester City) left footed shot from outside the box is saved in the bottom right corner. Assisted by Kyle Walker. Goal.  Brighton and Hove Albion 1, Manchester City 0. Glenn Murray (Brighton and Hove Albion) header from the left side of the six yard box to the bottom left corner. Assisted by Pascal Groß with a cross following a corner. Goal.  Brighton and Hove Albion 1, Manchester City 1. Sergio Agüero (Manchester City) left footed shot from a difficult angle on the left to the centre of the goal. Assisted by David Silva with a through ball. Attempt saved. Riyad Mahrez (Manchester City) right footed shot from a difficult angle on the right is saved in the centre of the goal. Assisted by David Silva. Goal.  Brighton and Hove Albion 1, Manchester City 2. Aymeric Laporte (Manchester City) header from very close range to the top left corner. Assisted by Riyad Mahrez with a cross following a corner. Attempt blocked. Riyad Mahrez (Manchester City) left footed

In [89]:
match_rank_df

url
http://www.premierleague.com/match/14040            Offside, Liverpool. Adam Lallana tries a throu...
http://www.premierleague.com/match/14041            Delay in match Simon Francis (Bournemouth) bec...
http://www.premierleague.com/match/14042            Foul by Federico Fernández (Swansea City). Att...
http://www.premierleague.com/match/14043            Diego Costa (Chelsea) wins a free kick in the ...
http://www.premierleague.com/match/14044            Attempt blocked. Salomón Rondón (West Bromwich...
                                                                          ...                        
https://www.espn.com/soccer/report?gameId=522802    Attempt missed. Piotr Zielinski (Napoli) left ...
https://www.espn.com/soccer/report?gameId=522804    Goal!  Inter Milan 1, Torino 0. Ivan Perisic (...
https://www.espn.com/soccer/report?gameId=522811    Attempt missed. Franck Kessié (Milan) right fo...
https://www.espn.com/soccer/report?gameId=522812    Attempt saved. Mehdi Boura

In [92]:
match_rank_df = rank.run()

Ranking events...


In [93]:
match_rank_df

,url,event
0,http://www.premierleague.com/match/14040,"Offside, Liverpool. Adam Lallana tries a throu..."
1,http://www.premierleague.com/match/14041,Delay in match Simon Francis (Bournemouth) bec...
2,http://www.premierleague.com/match/14042,Foul by Federico Fernández (Swansea City). Att...
3,http://www.premierleague.com/match/14043,Diego Costa (Chelsea) wins a free kick in the ...
4,http://www.premierleague.com/match/14044,Attempt blocked. Salomón Rondón (West Bromwich...
...,...,...
4518,https://www.espn.com/soccer/report?gameId=522802,Attempt missed. Piotr Zielinski (Napoli) left ...
4519,https://www.espn.com/soccer/report?gameId=522804,"Goal! Inter Milan 1, Torino 0. Ivan Perisic (..."
4520,https://www.espn.com/soccer/report?gameId=522811,Attempt missed. Franck Kessié (Milan) right fo...
4521,https://www.espn.com/soccer/report?gameId=522812,Attempt saved. Mehdi Bourabia (Sassuolo) right...


In [94]:
rank.path

'/home/carlos/MasterDS/tfm/models/baseline_rank/60099ac54d'

## Ejecución

In [98]:
key_events = ['goal', 'red_card', 'penalty']
lags = [1, 3, 5]
target_metric = 'rouge'
drop_teams = True
lemma = True
metric_params = {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'}
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}

In [99]:
ltr = LTRFeaturesTargets(target_metric=target_metric, 
                        key_events=key_events,
                        lags=lags,
                        metric_params=metric_params,
                        count_vec_kwargs=count_vec_kwargs,
                        drop_teams=drop_teams,
                        lemma=lemma)

Setting target metric to rouge


In [100]:
rank = BaselineRank(ltr=ltr, n=10)

Received LTR Features targets. Training and scoring are not necessary


In [101]:
rank.run()

Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/baseline_rank/60099ac54d/summaries.csv
